In [2]:
%load_ext autoreload
%autoreload 2
import copy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.features.games import GameFeatures
from src.utils import load_data_template, generate_data_template

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
data.sample()

,Season,team_a,team_b,in_target,game_set,a_win,DayNum
268,2014,1124,1166,True,0,True,139.0


In [10]:
game_feat = GameFeatures(default_lags=5)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season_against_opponent, fillna=0, per_game=True)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney_against_opponent, fillna=0, per_game=True)
data.sample()

,Season,team_a,team_b,in_target,game_set,a_win,DayNum,games_won_in_season_team_a_x,games_won_in_season_team_a_lag-1_x,games_won_in_season_team_a_lag-2_x,...,games_won_in_tourney_against_opponent_team_a_lag-1_y,games_won_in_tourney_against_opponent_team_a_lag-2_y,games_won_in_tourney_against_opponent_team_a_lag-3_y,games_won_in_tourney_against_opponent_team_a_lag-4_y,games_won_in_tourney_against_opponent_team_a_lag-5_y,games_won_in_tourney_against_opponent_team_b_lag-1_y,games_won_in_tourney_against_opponent_team_b_lag-2_y,games_won_in_tourney_against_opponent_team_b_lag-3_y,games_won_in_tourney_against_opponent_team_b_lag-4_y,games_won_in_tourney_against_opponent_team_b_lag-5_y
9913,1997,1390,1448,False,0,True,139.0,18,19.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Event data

In [37]:
import os
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [38]:
events = pd.read_csv('data/Events_2010.csv', nrows=1000)
print(events.shape)
events.head()

(1000, 11)


,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,1,2010,7,1143,1293,0,0,0,1143,600578,sub_in
1,2,2010,7,1143,1293,0,0,0,1143,600584,sub_in
2,3,2010,7,1143,1293,0,0,0,1143,600585,sub_in
3,4,2010,7,1143,1293,0,0,10,1143,600581,miss2_lay
4,5,2010,7,1143,1293,0,0,10,1143,600581,reb_off


In [40]:
events = generate_data_template(events, ['EventType'])
print(events.shape)
events.head()

(1000, 6)


,Season,team_a,team_b,a_win,DayNum,EventType
0,2010,1143,1293,True,7,sub_in
1,2010,1143,1293,True,7,sub_in
2,2010,1143,1293,True,7,sub_in
3,2010,1143,1293,True,7,miss2_lay
4,2010,1143,1293,True,7,reb_off


In [42]:
events = list()
file_names = [file_ for file_ in os.listdir('data/') if 'Events' in file_]
for file in file_names:
    events.append(pd.read_csv('data/' + file, nrows=1000))

events = pd.concat(events).reset_index(drop=True)
print(events.shape)
events.head()

(8000, 11)


,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,2502631,2011,7,1228,1414,0,0,23,1228,607151,miss3_jump
1,2502632,2011,7,1228,1414,0,0,23,1414,609791,reb_def
2,2502633,2011,7,1228,1414,0,0,48,1414,609785,miss2_jump
3,2502634,2011,7,1228,1414,0,0,48,1414,609791,reb_off
4,2502635,2011,7,1228,1414,0,2,53,1414,609791,made2_lay


In [43]:
le = LabelEncoder()
le.fit(events['EventType'])
events['EventNum'] = le.transform(events['EventType'])

In [44]:
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventNum
0,2502631,2011,7,1228,1414,0,0,23,1228,607151,miss3_jump,15
1,2502632,2011,7,1228,1414,0,0,23,1414,609791,reb_def,17
2,2502633,2011,7,1228,1414,0,0,48,1414,609785,miss2_jump,12
3,2502634,2011,7,1228,1414,0,0,48,1414,609791,reb_off,18
4,2502635,2011,7,1228,1414,0,2,53,1414,609791,made2_lay,7


In [45]:
ohe = OneHotEncoder()
ohe.fit(events['EventNum'].values.reshape([-1, 1]))
ohe_events = pd.DataFrame(ohe.transform(events['EventNum'].values.reshape([-1, 1])).toarray())
ohe_events.columns = le.classes_
ohe_events.head()

,assist,block,foul_pers,foul_tech,made1_free,made2_dunk,made2_jump,made2_lay,made2_tip,made3_jump,...,miss3_jump,reb_dead,reb_def,reb_off,steal,sub_in,sub_out,timeout,timeout_tv,turnover
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
events = pd.concat([events, ohe_events], axis=1)
events.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,...,miss3_jump,reb_dead,reb_def,reb_off,steal,sub_in,sub_out,timeout,timeout_tv,turnover
0,2502631,2011,7,1228,1414,0,0,23,1228,607151,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2502632,2011,7,1228,1414,0,0,23,1414,609791,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2502633,2011,7,1228,1414,0,0,48,1414,609785,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2502634,2011,7,1228,1414,0,0,48,1414,609791,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2502635,2011,7,1228,1414,0,2,53,1414,609791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
events.groupby(['EventTeamID', 'Season'])[le.classes_].sum()

assist  block  foul_pers  foul_tech  made1_free  \
EventTeamID Season                                                    
1102        2014      11.0    4.0       18.0        0.0        17.0   
1103        2014      10.0    5.0       16.0        0.0         9.0   
            2015      17.0    5.0       21.0        0.0        12.0   
1104        2013      14.0    4.0       16.0        0.0        14.0   
            2015      17.0    9.0       16.0        0.0        17.0   
            2016      13.0    6.0       21.0        0.0        15.0   
            2017      13.0    8.0       19.0        0.0        18.0   
1105        2016      10.0    1.0       25.0        0.0        10.0   
1107        2013      13.0    2.0       22.0        0.0         6.0   
            2017      16.0    3.0       22.0        0.0        20.0   
1108        2010       0.0    0.0        0.0        0.0         0.0   
1112        2012      12.0    2.0       21.0        0.0        23.0   
            2015       1.0    0.0        0.0        0.0         0.0   
            2016       2.0    0.0        2.0        0.0         2.0   
            2017       0.0    0.0        1.0        0.0         0.0   
1114        2013       4.0    3.0        6.0        0.0         6.0   
1119        2014      16.0    5.0       24.0        0.0        10.0   
1143        2010      15.0    1.0       14.0        0.0        22.0   
1157        2014       6.0    3.0       15.0        0.0        10.0   
            2017       7.0    2.0       22.0        0.0        10.0   
1182        2013       8.0    6.0       13.0        0.0        15.0   
1184        2012      11.0    1.0       18.0        0.0         4.0   
1198        2010      15.0    2.0       18.0        0.0        14.0   
1228        2011      17.0    8.0       22.0        0.0        13.0   
1244        2016       6.0    4.0       25.0        0.0        19.0   
1268        2011      15.0    8.0       13.0        0.0        20.0   
1277        2017       1.0    0.0        0.0        0.0         0.0   
1280        2012      16.0    6.0        9.0        0.0        12.0   
1291        2015       0.0    0.0        0.0        0.0         0.0   
1293        2010      11.0    3.0       24.0        0.0        10.0   
1314        2010      23.0    8.0       19.0        0.0        16.0   
1326        2010       3.0    1.0        0.0        0.0         0.0   
1334        2016       1.0    0.0        3.0        0.0         0.0   
1336        2017       9.0    8.0       23.0        0.0        18.0   
1355        2013       8.0    2.0       14.0        0.0         9.0   
1370        2011       9.0    1.0       23.0        0.0         9.0   
1385        2012       1.0    0.0        1.0        0.0         1.0   
1404        2013       4.0    0.0       12.0        0.0         4.0   
1406        2015       9.0    2.0       20.0        1.0        11.0   
1408        2016      11.0    7.0       21.0        0.0        16.0   
1414        2011      12.0    1.0       21.0        0.0        14.0   
1420        2015      13.0    0.0       19.0        0.0        14.0   
1434        2012      15.0    1.0       23.0        0.0        17.0   
1456        2012       1.0    1.0        1.0        0.0         2.0   

                    made2_dunk  made2_jump  made2_lay  made2_tip  made3_jump  \
EventTeamID Season                                                             
1102        2014           3.0         8.0        5.0        0.0        10.0   
1103        2014           0.0         2.0       15.0        0.0         7.0   
            2015           0.0         2.0       11.0        0.0        12.0   
1104        2013           0.0        16.0        3.0        0.0         6.0   
            2015           2.0         9.0       10.0        1.0         7.0   
            2016           5.0         7.0       11.0        2.0         4.0   
            2017           1.0         2.0        8.0        0.0        10.0   
1105        2016           1.0   

In [16]:
events['EventType'].unique()

array(['sub_in', 'miss2_lay', 'reb_off', 'made2_jump', 'miss2_jump',
       'assist', 'made3_jump', 'block', 'reb_def', 'foul_pers',
       'miss1_free', 'made1_free', 'miss3_jump', 'turnover', 'sub_out',
       'steal', 'made2_dunk', 'timeout_tv', 'made2_lay', 'timeout',
       'reb_dead', 'made2_tip', 'miss2_dunk', 'miss2_tip', 'foul_tech'], dtype=object)

In [ ]:
def create_target():
    target = pd.read_csv('data/SampleSubmissionStage1.csv').set_index('ID').drop('Pred', axis=1)
    target['Season'] = target.index.map(lambda i: i.split('_')[0])
    target['team_a'] = target.index.map(lambda i: i.split('_')[1])
    target['team_b'] = target.index.map(lambda i: i.split('_')[2])
    target['in_target'] = True
    
    ncaa_games = pd.read_csv('data/NCAATourneyCompactResults.csv')
    ncaa_games['team_a'] = ncaa_games[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) < int(t[1]) else t[1], axis=1)
    ncaa_games['team_b'] = ncaa_games[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) > int(t[1]) else t[1], axis=1)
    ncaa_games['a_win'] = ncaa_games['WTeamID'] == ncaa_games['team_a']
    ncaa_games['ID'] = ncaa_games['Season'].astype(str) + '_' + ncaa_games['team_a'].astype(str) + '_' + ncaa_games['team_b'].astype(str)
    ncaa_games = ncaa_games.set_index('ID')
    
    target = target.join(ncaa_games['a_win'], how='left')    
    return target

In [ ]:
target = create_target()
print target.shape
target.head()

In [ ]:
target_2.to_csv('data/target.csv')

In [ ]:
ncaa_games = pd.read_csv('data/NCAATourneyCompactResults.csv')
ncaa_games = ncaa_games.astype({
    'LTeamID': str,
    'WTeamID': str,
    'Season': str
})
ncaa_games['diff'] = ncaa_games['WScore'] - ncaa_games['LScore']
print(ncaa_games.shape)
ncaa_games.sample()

In [ ]:
regular_games = pd.read_csv('data/RegularSeasonCompactResults.csv')
regular_games = regular_games.astype({
    'LTeamID': str,
    'WTeamID': str,
    'Season': str
})
regular_games['diff'] = regular_games['WScore'] - regular_games['LScore']
print(regular_games.shape)
regular_games.sample()

In [ ]:
data = cp.deepcopy(ncaa_games)
data['team_a'] = data[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) < int(t[1]) else t[1], axis=1)
data['team_b'] = data[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) > int(t[1]) else t[1], axis=1)
data['a_win'] = data['WTeamID'] == data['team_a']
data = data[['Season', 'team_a', 'team_b', 'a_win']]
data = pd.concat([data, target.reset_index(drop=True)]).fillna(0).astype({'a_win': bool, 'in_target': bool})

lags = 5
for team, opponent_team in [('a', 'b'), ('b','a')]:
    
    
    # GAMES WON IN PAST TOURNAMENTS
    won_in_ncaa = ncaa_games\
        .groupby(['WTeamID', 'Season']).count()[['diff']]\
        .rename(columns={'diff': 'won_in_ncaa_{}'.format(team)})
    for l in range(1, lags+1):
        won_in_ncaa[
            'won_in_ncaa_{}_lag-{}'.format(team, l)] = won_in_ncaa.groupby('WTeamID')[
            'won_in_ncaa_{}'.format(team)
        ].shift(l).fillna(0)
    won_in_ncaa.drop('won_in_ncaa_{}'.format(team), inplace=True, axis=1)
    data = pd.merge(data, won_in_ncaa,
                    left_on=['team_{}'.format(team), 'Season'], right_index=True,
                    how='left').fillna(0)
    
    # GAMES WON IN PAST SEASONS
    won_in_season = regular_games\
        .groupby(['WTeamID', 'Season']).count()[['diff']]\
        .rename(columns={'diff': 'won_in_season_{}'.format(team)})
    for l in range(0, lags+1):
        won_in_season[
            'won_in_season_{}_lag-{}'.format(team, l)] = won_in_season.groupby('WTeamID')[
            'won_in_season_{}'.format(team)
        ].shift(l).fillna(0)
    won_in_season.drop('won_in_season_{}'.format(team), inplace=True, axis=1)
    data = pd.merge(data, won_in_season,
                    left_on=['team_{}'.format(team), 'Season'], right_index=True,
                    how='left').fillna(0)
    
    
    # TOURNAMENT GAMES WON AGAINST OPPONENT
    ncaa_wins_against_opponent = ncaa_games\
        .groupby(['Season', 'WTeamID', 'LTeamID']).count()[['diff']]\
        .rename(columns={'diff': 'ncaa_wins_against_opponent_{}'.format(team)})
    for l in range(1, lags+1):
        ncaa_wins_against_opponent[
            'ncaa_wins_against_opponent_{}_lag-{}'.format(team, l)
        ] = ncaa_wins_against_opponent.groupby(['WTeamID', 'LTeamID'])\
            ['ncaa_wins_against_opponent_{}'.format(team)].shift(l)
    ncaa_wins_against_opponent.drop('ncaa_wins_against_opponent_{}'.format(team), inplace=True, axis=1)
    data = pd.merge(data, ncaa_wins_against_opponent,
                    left_on=['Season', 'team_{}'.format(team), 'team_{}'.format(opponent_team)],
                    right_index=True,
                    how='left').fillna(0)
    
    
    # SEASON GAMES WON AGAINST OPPONENT
    season_wins_against_opponent = regular_games\
        .groupby(['Season', 'WTeamID', 'LTeamID']).count()[['diff']]\
        .rename(columns={'diff': 'season_wins_against_opponent_{}'.format(team)})
    for l in range(0, lags+1):
        season_wins_against_opponent[
            'season_wins_against_opponent_{}_lag-{}'.format(team, l)
        ] = season_wins_against_opponent.groupby(['WTeamID', 'LTeamID'])\
            ['season_wins_against_opponent_{}'.format(team)].shift(l)
    season_wins_against_opponent.drop('season_wins_against_opponent_{}'.format(team), inplace=True, axis=1)
    data = pd.merge(data, season_wins_against_opponent,
                    left_on=['Season', 'team_{}'.format(team), 'team_{}'.format(opponent_team)],
                    right_index=True,
                    how='left').fillna(0)
    
    
data = data.dropna()
data.head(5)

In [ ]:
data.shape

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data[~data.in_target.astype(bool)].drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target'], axis=1)
y = data[~data.in_target.astype(bool)][['a_win']]

kf = KFold(n_splits=8, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

In [ ]:
from xgboost import plot_importance

plt.figure(figsize=(20,4))
plot_importance(xgb, max_num_features=15)
plt.show()

In [ ]:
data.head()

In [ ]:
X_sub = data[data.in_target.astype(bool)].drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target'], axis=1)
y_sub = data[data.in_target.astype(bool)][['Season', 'team_a', 'team_b', 'a_win', 'in_target']]

xgb = XGBClassifier()
xgb.fit(X.values, y.values.reshape(-1))
preds = xgb.predict_proba(X_sub.values)

sub = cp.deepcopy(y_sub)
sub['Pred'] = preds[:,1]
sub['ID'] = sub['Season'].map(str) + '_' + sub['team_a'].map(str) + '_' + sub['team_b'].map(str)
sub.set_index('ID', inplace=True)
sub.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target'], axis=1, inplace=True)
sub.head()

In [ ]:
y_sub

In [ ]:
plt.figure(figsize=(20,4))
plt.hist(sub['Pred'], bins=100)
plt.show()

In [ ]:
sub.to_csv('submissions/xgb_baseline_2_corrected_lags_and_features.csv')

In [ ]:
### 

In [ ]:
target = pd.read_csv('data/SampleSubmissionStage1.csv').set_index('ID').drop('Pred', axis=1)
target['Season'] = target.index.map(lambda i: i.split('_')[0])
target['team_a'] = target.index.map(lambda i: i.split('_')[1])
target['team_b'] = target.index.map(lambda i: i.split('_')[2])
target['in_target'] = True
target.head()

In [ ]:
ncaa_games = pd.read_csv('data/NCAATourneyCompactResults.csv')
ncaa_games['team_a'] = ncaa_games[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) < int(t[1]) else t[1], axis=1)
ncaa_games['team_b'] = ncaa_games[['WTeamID', 'LTeamID']].apply(lambda t: t[0] if int(t[0]) > int(t[1]) else t[1], axis=1)
ncaa_games['a_win'] = ncaa_games['WTeamID'] == ncaa_games['team_a']
ncaa_games['ID'] = ncaa_games['Season'].astype(str) + '_' + ncaa_games['team_a'].astype(str) + '_' + ncaa_games['team_b'].astype(str)
#ncaa_games = ncaa_games[['Season', 'team_a', 'team_b', 'a_win']]
#ncaa_games = pd.concat([ncaa_games, target.reset_index(drop=True)]).fillna(0).astype({'a_win': bool, 'in_target': bool})
ncaa_games = ncaa_games.set_index('ID')
ncaa_games.head()

In [ ]:
np.nansum()

In [ ]:
submission = pd.read_csv('data/SampleSubmissionStage1.csv')
submission.head()

In [ ]:
target = target.join(ncaa_games['a_win']).drop('in_target', axis=1).dropna().sort_values(['Season', 'team_a', 'team_b'])

In [ ]:
target['a_win'] = target['a_win'] * 1
target.to_csv('data/target.csv')

In [ ]:
to_submit = pd.merge(submission, target.reset_index(), how='left', on='ID')
to_submit['Pred'] = to_submit['a_win']
to_submit = to_submit.fillna(0)[['ID', 'Pred']]
to_submit.head()

In [ ]:
(to_submit*1).to_csv('data/ground_truth_submission.csv', index=False)

In [ ]:
## Features

In [ ]:
from src.features.games import GameFeatures

In [ ]:
game_features = GameFeatures()

In [ ]:
game_features.season_games

## Neural net

In [ ]:
from keras import Sequential
from keras.layers import Dense

In [ ]:
train_data = data.loc[~data['in_target']]
train_data.head()

In [ ]:
print train_data['a_win'].shape
sum(train_data['a_win'])

In [ ]:
cols_to_drop = ['Season', 'a_win', 'in_target', 'team_a', 'team_b']

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=data.shape[1]-len(cols_to_drop), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

In [ ]:
# Fit the model
model.fit(train_data.drop(cols_to_drop, axis=1), train_data['a_win']*1, epochs=10, batch_size=10)

In [ ]:
predictions = model.predict(train_data.drop(cols_to_drop, axis=1))

In [ ]:
pd.Series(predictions.reshape(-1)).describe()

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(predictions, bins=100)
plt.show()

In [ ]:
# evaluate the model
scores = model.evaluate(data.drop('a_win', axis=1), data['a_win'])

### Tensorflow nn

In [246]:
%load_ext autoreload
%autoreload 2
from src.features.games import GameFeatures
from src.utils import load_data_template
from src.models.nn.ann import ANN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [247]:
game_feat = GameFeatures()
data = load_data_template()
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season_against_opponent, per_game=True, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney_against_opponent, per_game=True, fillna=0)
data = data.dropna()
data.sample(10)

,Season,team_a,team_b,in_target,game_set,a_win,DayNum,games_won_in_season_team_a,games_won_in_season_team_a_lag-1,games_won_in_season_team_a_lag-2,...,games_won_in_season_against_opponent_team_b,games_won_in_season_against_opponent_team_b_lag-1,games_won_in_season_against_opponent_team_b_lag-2,games_won_in_season_against_opponent_team_b_lag-3,games_won_in_tourney_against_opponent_team_a_lag-1,games_won_in_tourney_against_opponent_team_a_lag-2,games_won_in_tourney_against_opponent_team_a_lag-3,games_won_in_tourney_against_opponent_team_b_lag-1,games_won_in_tourney_against_opponent_team_b_lag-2,games_won_in_tourney_against_opponent_team_b_lag-3
10601,2008,1242,1424,False,0,True,138.0,30,30.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10481,2006,1133,1338,False,0,True,139.0,20,13.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7680,2017,1211,1462,True,0,True,145.0,32,25.0,31.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9345,1988,1210,1350,False,0,False,139.0,21,16.0,24.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10945,2013,1235,1326,False,0,False,139.0,22,22.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9863,1996,1153,1280,False,0,False,146.0,25,21.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10387,2005,1261,1412,False,0,False,136.0,20,18.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10683,2009,1338,1462,False,0,True,143.0,27,26.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10368,2004,1400,1462,False,0,False,144.0,23,22.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3048,2015,1173,1344,True,0,True,137.0,25,23.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [248]:
print(data.shape)
print(data.dropna().shape)

(2117, 35)
(2117, 35)


In [249]:
X = data.loc[~data.in_target].drop(['Season', 'a_win', 'in_target', 'team_a', 'team_b', 'DayNum'], axis=1)
Y = data.loc[~data.in_target, 'a_win']*1
X_eval = data.loc[data.in_target].drop(['Season', 'a_win', 'in_target', 'team_a', 'team_b', 'DayNum'], axis=1)
Y_eval = data.loc[data.in_target, 'a_win']*1

In [250]:
sum(Y)/len(Y)

0.5116279069767442

In [251]:
param_dict = {
    'input_dim': X.shape[1],
    'batch_size': 50,
    'hidden_units': [32],
    'eta': 0.01,
    'num_epochs': 500,
}
print(param_dict)

{'input_dim': 29, 'batch_size': 50, 'hidden_units': [32], 'eta': 0.01, 'num_epochs': 500}


In [254]:
ann = ANN(**param_dict)

In [255]:
ann.train(X, Y, X_eval, Y_eval, verbose=True)

Epoch: 000 train_loss=0.761 eval_loss=0.514
Epoch: 005 train_loss=0.382 eval_loss=0.455
Epoch: 010 train_loss=0.368 eval_loss=0.459
Epoch: 015 train_loss=0.399 eval_loss=0.407
Epoch: 020 train_loss=0.363 eval_loss=0.429
Epoch: 025 train_loss=0.350 eval_loss=0.447
Epoch: 030 train_loss=0.351 eval_loss=0.455
Epoch: 035 train_loss=0.361 eval_loss=0.505
Epoch: 040 train_loss=0.347 eval_loss=0.407
Epoch: 045 train_loss=0.343 eval_loss=0.407
Epoch: 050 train_loss=0.341 eval_loss=0.406
Epoch: 055 train_loss=0.340 eval_loss=0.404
Epoch: 060 train_loss=0.339 eval_loss=0.401
Epoch: 065 train_loss=0.339 eval_loss=0.397
Epoch: 070 train_loss=0.344 eval_loss=0.415
Epoch: 075 train_loss=0.338 eval_loss=0.442
Epoch: 080 train_loss=0.337 eval_loss=0.455
Epoch: 085 train_loss=0.337 eval_loss=0.464
Epoch: 090 train_loss=0.337 eval_loss=0.464
Epoch: 095 train_loss=0.340 eval_loss=0.470
Epoch: 100 train_loss=0.340 eval_loss=0.456
Epoch: 105 train_loss=0.345 eval_loss=0.451
Epoch: 110 train_loss=0.336 eval

(0.33330241501331331, 0.43034416)

In [56]:
X.shape

(11229, 29)

In [124]:
ann.layers

{'l_0': <tf.Tensor 'add:0' shape=(?, 32) dtype=float32>}

In [125]:
ann.weights

{'w_0': <tf.Variable 'w_0:0' shape=(28, 32) dtype=float32_ref>,
 'w_out': <tf.Variable 'w_out:0' shape=(32, 2) dtype=float32_ref>}

In [216]:
## Random search

In [272]:
from src.models.random_search import RandomSearch

In [273]:
rs = RandomSearch('nn', ANN, X, Y, X_eval, Y_eval, iterations=50)

In [274]:
performance = rs.search()

{'input_dim': 29, 'eta': 0.014844844844844845, 'dropout': 0.84684684684684686, 'hidden_units': [37, 113], 'batch_size': 108, 'num_epochs': 297, 'train_loss': 0.10907566712962258, 'eval_loss': 0.69390225}
{'input_dim': 29, 'eta': 0.045035035035035038, 'dropout': 0.03403403403403403, 'hidden_units': [107, 24], 'batch_size': 112, 'num_epochs': 401, 'train_loss': 1.1808460525103979, 'eval_loss': 7.878624}
{'input_dim': 29, 'eta': 0.034304304304304305, 'dropout': 0.073073073073073078, 'hidden_units': [24, 5], 'batch_size': 42, 'num_epochs': 429, 'train_loss': 0.72686848895890377, 'eval_loss': 0.69295365}
{'input_dim': 29, 'eta': 0.040070070070070067, 'dropout': 0.79379379379379378, 'hidden_units': [92], 'batch_size': 24, 'num_epochs': 359, 'train_loss': 25.268472939729698, 'eval_loss': 8.2394772}
{'input_dim': 29, 'eta': 0.021771771771771774, 'dropout': 0.24224224224224225, 'hidden_units': [42, 66], 'batch_size': 54, 'num_epochs': 223, 'train_loss': 0.43653480101514741, 'eval_loss': 0.69343

{'input_dim': 29, 'eta': 0.02033033033033033, 'dropout': 0.61761761761761758, 'hidden_units': [103], 'batch_size': 82, 'num_epochs': 448, 'train_loss': 0.12431301649023843, 'eval_loss': 0.40530008}
{'input_dim': 29, 'eta': 0.033943943943943941, 'dropout': 0.90590590590590592, 'hidden_units': [103], 'batch_size': 17, 'num_epochs': 403, 'train_loss': 49.915901759091618, 'eval_loss': 8.2394772}
{'input_dim': 29, 'eta': 0.032862862862862864, 'dropout': 0.7877877877877878, 'hidden_units': [109, 95, 55], 'batch_size': 29, 'num_epochs': 172, 'train_loss': 16.565774720290609, 'eval_loss': 8.2394772}
{'input_dim': 29, 'eta': 0.035505505505505505, 'dropout': 0.48748748748748749, 'hidden_units': [120, 83], 'batch_size': 36, 'num_epochs': 134, 'train_loss': 0.98336579567856264, 'eval_loss': 0.69299608}
{'input_dim': 29, 'eta': 0.025935935935935933, 'dropout': 0.91091091091091092, 'hidden_units': [71], 'batch_size': 88, 'num_epochs': 159, 'train_loss': 1.8794733773578298, 'eval_loss': 8.2394772}
{'

In [ ]:
performance.head()